In [66]:
import os
import pickle
import numpy as np
import pandas as pd
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity



In [73]:
import nltk
#Given an object named 'texts' (a list of tokenized texts, ie a list of lists of tokens)
#from gensim.test.utils import common_texts as texts

# Create a corpus from a list of lists of tokens
from gensim.corpora.dictionary import Dictionary

from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
nltk.download('stopwords')

texts=[]
file = open("TokenVieuxM.txt", "r")
lines = file.readlines()
file.close()

#print(stopwords.words())

for line in lines:
  line=line.strip()
  lt=line.split(",")
#Potential ill-character cleaning
  for i in range(len(lt)):
    lt[i]=lt[i].replace('[','')
    lt[i]=lt[i].replace(']','')
    lt[i]=lt[i].replace('"','')
    lt[i]=lt[i].replace('\n','')
    lt[i]=lt[i].replace(' ', '')
#End : Potential ill-characters cleaning
# print(lt)
  ltc=[word for word in lt if not word in stopwords.words()] # Remove Stopwords activated
#  print("C", ltc)
  texts.append(lt)

#for text in texts:
  #print(text)

#Here set the number of topics(to be changed if necessary)
nb=10
  
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]
#print(corpus)

# Print dictionnary # ***********
# for i in id2word:
#   print(i, id2word[i])

# Train the lda model on the corpus.
from gensim.models import LdaModel
lda = LdaModel(corpus, num_topics=nb)

# Print topic descrition
for i in range(0, nb-1):
  value=lda.get_topic_terms(i)
#  print(value)
  # print("Topic ", i) # *******************
  # for j in value:
  #   print(id2word[j[0]], " - P=", j[1])
  # print() # *******************

from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence= ', coherence_lda) # *******************************

# from gensim.utils import simple_preprocess


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/krtucho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
doc_topics = lda.get_document_topics(corpus)
# doc_topic = []
# for topic in doc_topics:
#     for item in topic:
#         doc_topic.append(item)

# # doc_topics = [inside_doc_topic for inside_doc_topic in [doc_topic for doc_topic in doc_topics]]

doc_topics

In [91]:
# Get the topic distribution for each document

most_typical_docs = {}
for i, doc_topic in enumerate(doc_topics):
    for item in doc_topic: 
        topic, prob = item
        if most_typical_docs.__contains__(topic):
            if most_typical_docs[topic][1] < prob:
                most_typical_docs[topic] = (topic, prob, texts[i])
        else:
            most_typical_docs[topic] = (topic, prob, texts[i])
        
print("Typical Document for each topic, topic proportion (or probability) in documents") 
for i, prob, doc in most_typical_docs.values():
    print(f"Topic {i}: {doc} \nwith prob: {prob}")
    print()

Typical Document for each topic, topic proportion (or probability) in documents
Topic 0: ['abstract', 'acoustic', 'signals', 'are', 'extensively', 'used', 'for', 'guiding', 'various', 'behaviors', 'in', 'frogs', 'such', 'as', 'vocalization', 'and', 'phonotaxis', 'while', 'numerous', 'studies', 'have', 'investigated', 'the', 'anatomy', 'and', 'physiology', 'of', 'the', 'auditory', 'system', 'our', 'knowledge', 'of', 'intrinsic', 'properties', 'and', 'connectivity', 'of', 'individual', 'auditory', 'neurons', 'remains', 'poor', 'moreover', 'the', 'neural', 'basis', 'of', 'audiomotor', 'integration', 'still', 'has', 'to', 'be', 'elucidated', 'we', 'determined', 'basic', 'response', 'patterns', 'dendritic', 'arborization', 'and', 'axonal', 'projection', 'patterns', 'of', 'auditory', 'midbrain', 'units', 'with', 'intracellular', 'recording', 'and', 'staining', 'techniques', 'in', 'an', 'isolated', 'brain', 'preparation', 'the', 'subnuclei', 'of', 'the', 'torus', 'semicircularis', 'subserve',

In [85]:
# Get the topic distribution for each document
doc_topics = lda.get_document_topics(corpus)

# print(doc_topics)
# doc_topics
# for doc_topic in doc_topics:
#     print(doc_topic)

# Get the most typical document for each topic based on cosine similarity
topic_words = [lda.get_topic_terms(i, topn=10) for i in range(nb)]
doc_vectors = np.zeros((len(texts), nb))
for i, doc in enumerate(texts):
    doc_term_vec = id2word.doc2bow(doc)
    for j, prob in lda[doc_term_vec]:
        doc_vectors[i][j] = prob
        # print("Prob: ", prob)



most_typical_docs = []
for topic in range(nb):
    topic_vec = np.zeros((1, nb))
    for word, _ in topic_words[topic]:
        if word in id2word.token2id:
            topic_vec[0][lda.get_term_topics(word)[0][0]] = 1
    similarity_scores = cosine_similarity(doc_vectors, topic_vec)
    # print("sim Scores: ", similarity_scores)
    most_typical_doc_index = np.argmax(similarity_scores)
    # print("topic: ", topic, "mtdi: ", most_typical_doc_index)
    most_typical_docs.append(texts[most_typical_doc_index])

# Print the most typical document for each topic
print("Typical Document for each topic, cosine_similarity") 
for i, doc in enumerate(most_typical_docs):
    print(f"Topic {i}: {doc}")
    print()

Typical Document for each topic, cosine_similarity
Topic 0: ['abstract', 'the', 'accumulation', 'of', 'nonenzymatic', 'modifications', 'on', 'both', 'dna', 'and', 'protein', 'molecules', 'under', 'the', 'attack', 'of', 'reactive', 'oxygen', 'species', 'ros', 'is', 'one', 'of', 'the', 'most', 'possible', 'factors', 'responsible', 'for', 'the', 'functional', 'deterioration', 'in', 'aged', 'cells', 'direct', 'protein', 'modifications', 'as', 'well', 'as', 'dna', 'damages', 'may', 'be', 'detectable', 'in', 'part', 'by', 'proteome', 'analysis', 'if', 'the', 'gene', 'expression', 'is', 'affected', 'by', 'the', 'damages', 'on', 'dna', 'the', 'novel', 'term', '“proteome”', 'which', 'is', 'a', 'compound', 'of', '“protein”', 'and', '“genome”', 'means', 'a', 'whole', 'set', 'of', 'proteins', 'expressed', 'in', 'a', 'tissue', 'or', 'a', 'cell', 'strain', 'to', 'be', 'investigated', 'proteomics', 'is', 'a', 'methodology', 'for', 'analyzing', 'proteomes', 'in', 'proteomics', 'twodimensional', 'gel',

In [88]:
topic_counts = {}
sums = 0
for doc_topic in doc_topics:
    for item in doc_topic: 
        topic, prob = item
        if topic_counts.__contains__(str(topic)):
            topic_counts[str(topic)] = topic_counts[str(topic)]+1
        else:
            topic_counts[str(topic)] = 1
        sums += 1

rounds = [(topic, round(topic_count/sums, 4)) for (topic, topic_count) in topic_counts.items()]
print(rounds)

[('0', 0.3333), ('7', 0.1667), ('5', 0.5)]
